# Getting data

In order to obtain our data regarding crypto currencies we will use Messari's API so that it provides us with up to date information on the matter.

### REST API

We will request Messari.io for top assets and their market price back one year. For that we will need to first obtain the assets and then ask for their historic data.

In [1]:
import requests, json

url = "https://data.messari.io/api/v1/assets?fields=id,slug,symbol,metrics/market_data/price_usd"
response = requests.get(url)
assets = json.loads(response.text)['data']

Now that we know our assets we can request their history.

In [2]:
from datetime import date
import pandas as pd

for a in assets:
    asset = a['slug']
    today = date.today()
    yearago = today.replace(year = today.year -1)
    url = f'https://data.messari.io/api/v1/assets/{asset}/metrics/price/time-series?start={yearago}&end={today}&interval=1d'
    response = requests.get(url)
    info = json.loads(response.text)
    if "data" in info:
        columns = info["data"]["parameters"]["columns"]
        data = info["data"]["values"]
        df = pd.DataFrame(data, columns=columns)
        df["slug"] = asset
        df.to_csv(f"../data/{today}-{asset}.csv")

In [41]:
df.head()

,timestamp,open,high,low,close,volume,slug
0,1609977600000,6.430649,6.831946,5.902719,6.264813,2.949064e+08,uniswap
1,1610064000000,6.266868,6.407123,5.447225,5.883652,2.875378e+08,uniswap
2,1610150400000,5.876753,6.369207,5.668347,6.248685,2.118909e+08,uniswap
3,1610236800000,6.276470,6.608679,5.504691,5.983117,2.599493e+08,uniswap
4,1610323200000,5.988630,5.999697,4.394917,5.244629,4.099609e+08,uniswap


The information contains the timestamp; open, highest, lowest and closing prices in USD and the volume of operations.